# [Python&数理最適化を用いて最適な栄養素の献立を算出するレシピ](https://axross-recipe.com/recipes/097)

In [ ]:
# GC mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# パッケージのダウンロード
!pip install pulp

In [ ]:
# データのダウンロード（あらかじめDLしておく）
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/food.csv" /content/
!cp "/content/drive/MyDrive/Colab Notebooks/note_Axross/_data/nutrient.csv" /content/

In [ ]:
# パッケージのデモ（pulpを用いて線形計画法を解く）
from pulp import *

problem = LpProblem('simple_problem', LpMaximize)
x = LpVariable('x')
y = LpVariable('y')

problem += x + y

problem += -4*x + 12 >= y
problem += -1/2*x+4 >= y

status = problem.solve()
print(LpStatus[status])

print("x:",x.value())
print("y:",y.value())

In [ ]:
# dataset読込（当該食料の1ドルあたりの価格・栄養素）
import pandas as pd
dffoods = pd.read_csv("food.csv")
dffoods

In [ ]:
# dataset読込（1日あたりに必要な栄養素）
dfnutrient_lowerlimit = pd.read_csv("nutrient.csv")
dfnutrient_lowerlimit

In [ ]:
# 関数を設定
problem = LpProblem('Stiger Diet', LpMinimize)

# food_amount、食料xに何ドル使うか？という変数のリストです。これから線形最適化を使い、これら変数の最適な値を算出することになります。
# food_used[0]はWheat Flour, food_used[1]はMacaroni...という形で対応します。
# また、必要ではないですが、lowBound=0を入れて、これら変数は必ず0以上であるという条件を追加しています。
food_amount = [LpVariable('food_{0}'.format(i), lowBound=0) for i in range(len(dffoods))]

# 目的関数を設定しています。lpSumは1つのリストの総和を求めるための関数です。（numpyのsumを使っても問題ないです。）
# food_amountはその食料に何ドル使うか？の変数なので、単純に和をとれば食料費の総額になります。
problem += lpSum(food_amount)

In [ ]:
# 制約の設定（lpDotは2つのリストの内積を求める関数。購入総額が目的関数に設定）
for k,v in dfnutrient_lowerlimit.iterrows():
    problem += lpDot(food_amount, dffoods[v.Nutrient]) >= v.Intake

In [ ]:
# 最適化
status = problem.solve()
print(LpStatus[status])

# 年間コストを算出するため、最適値に365を乗じていっています。
annual_cost = 0
for i, fa in enumerate(food_amount):
    if fa.value() > 0:
        print("{0} : ${1}".format(dffoods.food[i], fa.value()))
        annual_cost += 365 * fa.value()
        
print("上記がコスト最小となる最適な食材の組み合わせで、年間{0}ドルで済む".format(annual_cost))